# Notebook para ejecutar NSForest_by

## Import libreries

In [1]:
from NSForest_v3dot9_2_test_PreBinarySelect import *

## Load data

In [2]:
# Loading h5ad
file = "/home/a/Escritorio/TFM/src/NSForest/data/data_final_selection/adata_switched_2_2.h5ad"
adata = sc.read_h5ad(file)
adata_test = sc.read_h5ad("/home/a/Escritorio/TFM/src/NSForest/data/data_final_selection/adata_selected_park_6cl.h5ad")

## Quick view of datasets

In [3]:
adata #quick look of the data

AnnData object with n_obs × n_vars = 85429 × 16653
    obs: 'Origin', 'suspension_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'author_cell_type', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'development_stage_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'donor_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'batch'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'schema_version', 'title'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

In [4]:
adata_test

AnnData object with n_obs × n_vars = 20571 × 16653
    obs: 'Origin', 'suspension_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'author_cell_type', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'development_stage_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'donor_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'schema_version', 'title'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

## Get cluster names

In [5]:
cluster_header = "cell_type" #<---
medians_header = "medians_" + cluster_header
binary_scores_header = "binary_scores_" + cluster_header

In [6]:
np.unique(adata.obs[cluster_header])

array(['endothelial cell', 'epithelial cell of proximal tubule',
       'kidney collecting duct principal cell',
       'kidney connecting tubule epithelial cell',
       'kidney distal convoluted tubule epithelial cell',
       'renal beta-intercalated cell'], dtype=object)

In [7]:
len(np.unique(adata.obs[cluster_header]))

6

In [8]:
filename = file.replace(".h5ad", "_preprocessed.h5ad")
print(f"Saving new anndata object as...\n{filename}")
adata.write_h5ad(filename)

Saving new anndata object as...
/home/a/Escritorio/TFM/src/NSForest/data/data_final_selection/adata_switched_2_2_preprocessed.h5ad


In [9]:
cluster_header = "cell_type"
medians_header = "medians_" + cluster_header
binary_scores_header = "binary_scores_" + cluster_header
adata = sc.read_h5ad(file.replace(".h5ad", "_preprocessed.h5ad"))
# Extrae el nombre del archivo sin la extensión
dataset_name = os.path.splitext(os.path.basename(file))[0]
outputfilename = "RF_by"
output_folder = f"../outputs_experimentation/RF_by/{dataset_name}/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Creating new directory...\n{output_folder}")

## Runing NSForest to obtain the biomarkers

In [10]:
adata = prep_medians(adata, cluster_header, use_mean = False, positive_genes_only = True) 
adata = prep_binary_scores(adata, cluster_header, medians_header)

Calculating medians...


Calculating medians (means) per cluster: 100%|██████████| 6/6 [01:03<00:00, 10.52s/it]


Saving calculated medians as adata.varm.medians_cell_type
--- 63.12512135505676 seconds ---
Only positive genes selected. 358 positive genes out of 16653 total genes
Calculating binary scores...


Calculating binary scores per cluster: 100%|██████████| 6/6 [00:00<00:00,  8.61it/s]

Saving calculated binary scores as adata.varm.binary_scores_cell_type
--- 0.7054269313812256 seconds ---
median: 0.0
mean: 0.2188787971406454
std: 0.36592288050865884


### Runing NSForest using all clusters

In [11]:
NSForest_results = NSForest(adata, cluster_header, medians_header, binary_scores_header, beta = 0.2,  n_trees = 100,
                   gene_selection="BinaryFirst_high",
                   output_folder = output_folder, outputfilename_prefix = outputfilename)

Preparing data...
Pre-selecting genes based on binary scores...
	 Threshold (mean + 2 * std): 0.9507245581579631
	Average number of genes after gene_selection in each cluster: 36.666666666666664
Saving number of genes selected per cluster as...
../outputs_experimentation/RF_by/adata_switched_2_2/RF_by_gene_selection.csv
Number of clusters to evaluate: 6


running NSForest on all clusters:   0%|          | 0/6 [00:00<?, ?it/s]

1 out of 6:
	endothelial cell
	 Pre-selected 22 genes to feed into Random Forest.
Tiempo de ejecución: 5.83 segundos


running NSForest on all clusters:  17%|█▋        | 1/6 [00:11<00:55, 11.07s/it]

	['ENSMUSG00000021759', 'ENSMUSG00000039706']
	fbeta: 0.952259847093361
2 out of 6:
	renal beta-intercalated cell
	 Pre-selected 47 genes to feed into Random Forest.
Tiempo de ejecución: 7.50 segundos


...

	['ENSMUSG00000054640', 'ENSMUSG00000028030']
	fbeta: 0.8254410897195259
5 out of 6:
	kidney distal convoluted tubule epithelial cell
	 Pre-selected 84 genes to feed into Random Forest.
Tiempo de ejecución: 18.35 segundos


running NSForest on all clusters:  83%|████████▎ | 5/6 [01:12<00:16, 16.12s/it]

	['ENSMUSG00000031766', 'ENSMUSG00000028017']
	fbeta: 0.9357226100224669
6 out of 6:
	kidney collecting duct principal cell
	 Pre-selected 4 genes to feed into Random Forest.
Tiempo de ejecución: 2.32 segundos
	Only 4 out of 15 top Random Forest features with median > 0 will be further evaluated.


running NSForest on all clusters: 100%|██████████| 6/6 [01:16<00:00, 12.75s/it]

	['ENSMUSG00000023013', 'ENSMUSG00000004988']
	fbeta: 0.8982900517728595
Saving supplementary table as...
../outputs_experimentation/RF_by/adata_switched_2_2/RF_by_supplementary.csv
Saving markers table as...
../outputs_experimentation/RF_by/adata_switched_2_2/RF_by_markers.csv
Saving results table as...
../outputs_experimentation/RF_by/adata_switched_2_2/RF_by_results.csv
Saving final results table as...
../outputs_experimentation/RF_by/adata_switched_2_2/RF_by_results.csv
Tiempo total de la ejecución: 76.61 segundos


In [12]:
NSForest_results

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,thresholds,binary_genes
0,endothelial cell,4824,0.952260,0.990658,80583,22,2491,2333,2,"[ENSMUSG00000021759, ENSMUSG00000039706]","[0.8481884002685547, 0.7238551378250122]","[ENSMUSG00000027210, ENSMUSG00000021759, ENSMU..."
1,renal beta-intercalated cell,1756,0.956284,0.988285,83662,11,828,928,2,"[ENSMUSG00000028238, ENSMUSG00000020566]","[0.4547166973352432, 0.4649316668510437]","[ENSMUSG00000024485, ENSMUSG00000028238, ENSMU..."
2,epithelial cell of proximal tubule,59680,0.935614,0.963448,24519,1230,27259,32421,3,"[ENSMUSG00000030945, ENSMUSG00000022613, ENSMU...","[0.40984900295734406, 0.3071601241827011, 0.20...","[ENSMUSG00000030945, ENSMUSG00000022613, ENSMU..."
3,kidney connecting tubule epithelial cell,3072,0.825441,0.950241,82326,31,2480,592,2,"[ENSMUSG00000054640, ENSMUSG00000028030]","[2.118518114089966, 1.3690828680992126]","[ENSMUSG00000054640, ENSMUSG00000028030, ENSMU..."
4,kidney distal convoluted tubule epithelial cell,11491,0.935723,0.966446,73730,208,5500,5991,2,"[ENSMUSG00000031766, ENSMUSG00000028017]","[1.1833228468894958, 1.0618106722831726]","[ENSMUSG00000031766, ENSMUSG00000028017, ENSMU..."
5,kidney collecting duct principal cell,4606,0.898290,0.954416,80743,80,2931,1675,2,"[ENSMUSG00000023013, ENSMUSG00000004988]","[1.4653971195220947, 1.1409903764724731]","[ENSMUSG00000023013, ENSMUSG00000004988, ENSMU..."
6,Average,14238,0.917269,0.968916,70927,263,6914,7323,-,-,-,-


### Testing the model performance:

### Approach 1:
- Train a decission tree for each of selected genes as biomarkers by NSForest
- Obtein the prediction as the dot product of the individual prediction for each gene

In [13]:
NSForest_results = NSForest_results.iloc[:-1]
df_test_result_total = myDecisionTreeEvaluationTest(adata, adata_test, cluster_header, NSForest_results, beta = 0.2,
                                                    output_folder = output_folder, outputfilename_prefix = outputfilename)

In [14]:
df_test_result_total

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,threshold
0,endothelial cell,638,0.271147,1.000000,19933,0,629,9,2,"['ENSMUSG00000021759', 'ENSMUSG00000039706']","[0.8481884002685547, 0.7238551378250122]"
1,renal beta-intercalated cell,691,0.903678,0.973799,19874,6,468,223,2,"['ENSMUSG00000028238', 'ENSMUSG00000020566']","[0.4547166973352432, 0.4649316668510437]"
2,epithelial cell of proximal tubule,13152,0.906059,0.964708,7246,173,8423,4729,3,"['ENSMUSG00000030945', 'ENSMUSG00000022613', '...","[0.40984900295734406, 0.3071601241827011, 0.20..."
3,kidney connecting tubule epithelial cell,831,0.000000,0.000000,19740,0,831,0,2,"['ENSMUSG00000054640', 'ENSMUSG00000028030']","[2.118518114089966, 1.3690828680992126]"
4,kidney distal convoluted tubule epithelial cell,4890,0.911411,0.958373,15594,87,2887,2003,2,"['ENSMUSG00000031766', 'ENSMUSG00000028017']","[1.1833228468894958, 1.0618106722831726]"
5,kidney collecting duct principal cell,369,0.937311,0.957627,20192,10,143,226,2,"['ENSMUSG00000023013', 'ENSMUSG00000004988']","[1.4653971195220947, 1.1409903764724731]"
6,Average,3428,0.654934,0.809085,17096,46,2230,1198,-,-,-


### Approach 2:
- Train a decission tree with all NSForest selected gene at the same time

In [15]:
df_combined_test_result = myDecisionTreeEvaluationTestCombined(adata, adata_test, cluster_header, NSForest_results, beta = 0.2,
                                                               output_folder = output_folder, outputfilename_prefix = outputfilename)

In [16]:
df_combined_test_result

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,threshold
0,endothelial cell,638,0.662252,0.943396,19930,3,588,50,2,"['ENSMUSG00000021759', 'ENSMUSG00000039706']",[0.8481884002685547]
1,renal beta-intercalated cell,691,0.707209,0.714815,19726,154,305,386,2,"['ENSMUSG00000028238', 'ENSMUSG00000020566']",[0.4547166973352432]
2,epithelial cell of proximal tubule,13152,0.828174,0.832054,5450,1969,3397,9755,3,"['ENSMUSG00000030945', 'ENSMUSG00000022613', '...",[0.40984900295734406]
3,kidney connecting tubule epithelial cell,831,0.405983,0.423469,19514,226,665,166,2,"['ENSMUSG00000054640', 'ENSMUSG00000028030']",[2.118518114089966]
4,kidney distal convoluted tubule epithelial cell,4890,0.833702,0.845893,15135,546,1893,2997,2,"['ENSMUSG00000031766', 'ENSMUSG00000028017']",[1.1833228468894958]
5,kidney collecting duct principal cell,369,0.711595,0.709924,20088,114,90,279,2,"['ENSMUSG00000023013', 'ENSMUSG00000004988']",[1.4653971195220947]
6,Average,3428,0.691486,0.744925,16640,502,1156,2272,-,-,-


In [17]:
new_eval = myDecisionTreeEvaluationTestMove(adata, adata_test, cluster_header, NSForest_results, beta = 0.2,
                                                               output_folder = output_folder, outputfilename_prefix = outputfilename, coef=0.5)

In [18]:
new_eval

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,original_threshold,new_threshold
0,endothelial cell,638,0.223862,1.000000,19933,0,631,7,2,"['ENSMUSG00000021759', 'ENSMUSG00000039706']","[0.8481884002685547, 0.7238551378250122]","[0.884131882339716, 0.7579623945057392]"
1,renal beta-intercalated cell,691,0.896081,0.980100,19876,4,494,197,2,"['ENSMUSG00000028238', 'ENSMUSG00000020566']","[0.4547166973352432, 0.4649316668510437]","[0.5515786409378052, 0.5622370913624763]"
2,epithelial cell of proximal tubule,13152,0.885052,0.975062,7329,90,9633,3519,3,"['ENSMUSG00000030945', 'ENSMUSG00000022613', '...","[0.40984900295734406, 0.3071601241827011, 0.20...","[0.824158564209938, 0.7460954636335373, 0.4970..."
3,kidney connecting tubule epithelial cell,831,0.000000,0.000000,19740,0,831,0,2,"['ENSMUSG00000054640', 'ENSMUSG00000028030']","[2.118518114089966, 1.3690828680992126]","[2.376619905233383, 1.4482608661055565]"
4,kidney distal convoluted tubule epithelial cell,4890,0.887445,0.976401,15649,32,3566,1324,2,"['ENSMUSG00000031766', 'ENSMUSG00000028017']","[1.1833228468894958, 1.0618106722831726]","[1.6282113194465637, 1.4157971143722534]"
5,kidney collecting duct principal cell,369,0.938741,0.963470,20194,8,158,211,2,"['ENSMUSG00000023013', 'ENSMUSG00000004988']","[1.4653971195220947, 1.1409903764724731]","[1.6408644318580627, 1.2931244373321533]"
6,Average,3428,0.638530,0.815839,17120,22,2552,876,-,-,-,-
